In [1]:
from multitask_dataset import SingerMultiTaskDataset
from torch.utils.data import DataLoader
from torch.optim import Adam
from models import HuBERTMultiHead
import pandas as pd

In [2]:
train_audio_folder = '/media/maximos/9C33-6BBD/data/melos_singers/Rebetika_vowels/train/'
csv_path = '/media/maximos/9C33-6BBD/data/melos_singers/features/multitask_targets.csv'

In [3]:
feats = pd.read_csv(csv_path, delimiter=',')
features_list = list(feats.columns)
print(features_list)
del(features_list[:2])
print(features_list)

['Unnamed: 0', 'names', 'singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', 'F2_Hz', 'F3_HZ', 'F4_Hz', 'Rate', 'Depth', 'Regularity']
['singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', '

In [4]:
task_labels_num_out = {}
for i in range(1, len(features_list)-3, 1):
    task_labels_num_out[features_list[i]] = 1
# add singer identification
task_labels_num_out['singer_id'] = feats['singer_id'].max()+1 # accounting for zero
print(task_labels_num_out)

{'Pitch': 1, 'SpectralCentroid': 1, 'SpectralSpread': 1, 'SpectralSkewness': 1, 'SpectralKurtosis': 1, 'SpectralFlatness': 1, 'SpectralCrest': 1, 'SpectralSlope': 1, 'SpectralDecrease': 1, 'SpectralRollOff': 1, 'SpectralVariation': 1, 'SpectralFlux': 1, 'HarmonicSpectralDeviation': 1, 'Tristimulus_1': 1, 'Tristimulus_2': 1, 'Tristimulus_3': 1, 'HarmonicOddToEvenRatio': 1, 'Inharmonicity': 1, 'HarmonicEnergy': 1, 'NoiseEnergy': 1, 'Noisiness': 1, 'HarmonicToNoiseEnergy': 1, 'PartialsToNoiseEnergy': 1, 'F1_Hz': 1, 'F2_Hz': 1, 'F3_HZ': 1, 'F4_Hz': 1, 'singer_id': 6}


In [5]:
model = HuBERTMultiHead(task_labels_num_out=task_labels_num_out)

You are using a model of type hubert to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
training_data = SingerMultiTaskDataset(train_audio_folder, csv_path)

In [7]:
print(len(training_data))

336


In [8]:
print(training_data[235])

{'input_values': array([ 9.6231786e-05, -4.8065151e-04, -1.0562916e-03, ...,
        2.1723036e-02,  3.0537494e-02,  3.0633982e-02], dtype=float32), 'labels': {'Unnamed: 0': 235, 'names': 'keti_grei_e_7', 'singer_id': 3, 'Pitch': 2.133238311537819, 'SpectralCentroid': 0.8911490491285525, 'SpectralSpread': 0.5403551019520066, 'SpectralSkewness': -0.1632347826443252, 'SpectralKurtosis': -0.17152970573613813, 'SpectralFlatness': -0.15833463048318755, 'SpectralCrest': -0.3623104746123098, 'SpectralSlope': 0.34877741431176684, 'SpectralDecrease': -0.784948891058209, 'SpectralRollOff': -0.853288380165642, 'SpectralVariation': 1.4472345812459628, 'SpectralFlux': -0.5510414596247086, 'HarmonicSpectralDeviation': 1.1137349506141108, 'Tristimulus_1': 0.3715199337662464, 'Tristimulus_2': 0.5525438422866596, 'Tristimulus_3': -0.7352574445388284, 'HarmonicOddToEvenRatio': -0.3191256636083456, 'Inharmonicity': -0.9570538326469452, 'HarmonicEnergy': -0.2090866958290402, 'NoiseEnergy': 0.3527717537887

In [9]:
dataloader = DataLoader(training_data, batch_size=8, shuffle=True, collate_fn=model.collate_fn)

In [10]:
print(dataloader.__dict__)

{'dataset': <multitask_dataset.SingerMultiTaskDataset object at 0x7dc61abd7f50>, 'num_workers': 0, 'prefetch_factor': None, 'pin_memory': False, 'pin_memory_device': '', 'timeout': 0, 'worker_init_fn': None, '_DataLoader__multiprocessing_context': None, '_dataset_kind': 0, 'batch_size': 8, 'drop_last': False, 'sampler': <torch.utils.data.sampler.RandomSampler object at 0x7dc5a5a527b0>, 'batch_sampler': <torch.utils.data.sampler.BatchSampler object at 0x7dc59c4bae70>, 'generator': None, 'collate_fn': <bound method HuBERTMultiHead.collate_fn of HuBERTMultiHead(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      

In [11]:
b = next(iter(dataloader))

In [12]:
print(len(b[0]['input_values']))
print(len(b[0]['attention_mask']))
print(len(b[1]))
print(b[1])

8
8
33
{'Unnamed: 0': [27, 177, 135, 52, 35, 305, 6, 180], 'names': ['bellou_w_3', 'kazantzidis_old_w_8', 'kazantzidis_old_e_10', 'bithikotsis_i_4', 'bithikotsis_a_2', 'tsaousakis_a_16', 'bellou_a_7', 'kazantzidis_u_2'], 'singer_id': [0, 2, 2, 1, 1, 5, 0, 2], 'Pitch': [0.1642618821017367, -0.9645244655609191, 0.5205380897386542, -0.017097976231618465, -0.08083967680093286, -0.6119452503436076, -0.6030482973860293, 0.03168414987208851], 'SpectralCentroid': [-0.6706910222963102, -1.3931127566485852, -0.4445704536184175, 1.0011995007507617, 1.2060898924123618, 0.5473692034352133, 0.7607217467458218, -0.5602795240706216], 'SpectralSpread': [-1.0128964784165524, -0.94580835730321, -0.5400771238192227, 0.849711322477832, 1.095975987089599, 0.5883087952879235, 0.5451649593183291, 1.2837711779639127], 'SpectralSkewness': [1.2504636364487018, 1.58363921493489, 0.026814286987436466, -1.53757775195964, -0.0695002384265383, 0.06750122096956424, -0.5315402897252111, 1.9308555984962907], 'SpectralKu

In [13]:
model.train()
optimizer = Adam(model.parameters())

In [20]:
y = model(
    audio_normalized=b[0]['input_values'],
    attention_mask=b[0]['attention_mask'],
    labels=b[1],
    output_attentions=False,
    output_hidden_states=False,
    return_dict=True
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 1.95 GiB of which 14.62 MiB is free. Including non-PyTorch memory, this process has 1.93 GiB memory in use. Of the allocated memory 1.83 GiB is allocated by PyTorch, and 20.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [15]:
print(y)

SequenceClassifierOutput(loss=tensor(0.2798, device='cuda:0', dtype=torch.float16,
       grad_fn=<MseLossBackward0>), logits=tensor([[-0.0862],
        [ 0.0374],
        [-0.1012],
        [ 0.0223],
        [ 0.0036],
        [ 0.0132],
        [ 0.0131],
        [-0.0529]], device='cuda:0', dtype=torch.float16,
       grad_fn=<AddmmBackward0>), hidden_states=tensor([[[ 4.0192e-02,  5.6458e-02,  8.1787e-02,  ..., -3.1299e-01,
          -7.3669e-02,  1.6094e+00],
         [ 1.8506e-01, -1.5564e-01,  1.4624e-01,  ..., -3.0420e-01,
          -2.7026e-01,  1.2051e+00],
         [ 3.2163e-04,  1.1932e-01,  1.6504e-01,  ..., -2.5317e-01,
          -2.5049e-01,  1.2812e+00],
         ...,
         [-7.4097e-02, -2.5049e-01, -5.1544e-02,  ..., -7.6355e-02,
          -1.3135e-01, -5.9766e-01],
         [ 2.6184e-02, -1.5442e-01,  2.4429e-02,  ..., -1.4087e-01,
          -9.2224e-02, -6.5039e-01],
         [ 6.7978e-03,  1.8384e-01, -1.7590e-01,  ..., -9.5032e-02,
          -2.1149e-02, -5.54

In [16]:
optimizer.zero_grad()
y.loss.backward(retain_graph=True)
optimizer.step()

In [21]:
print(y.loss.grad)

None


/tmp/ipykernel_17553/1258627354.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(y.loss.grad)
